In [1]:
import os
import mne
import pyedflib
import wfdb
import torch
import torchvision
import numpy as np
import pandas as pd
import shutil
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.init as init
import json
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from PIL import Image
from torchvision import transforms
from torch.utils.tensorboard import SummaryWriter

# 1.获取文件路径(使用时需要修改数据集根目录)

In [2]:
#数据集根目录
root_dir = "D:\\eeg-motor-movementimagery-dataset-1.0.0\\files"

#获取所有受试者的文件路径
subject_dirs_list = [os.path.join(root_dir, subject) for subject in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, subject))]

#分别获取EEG信号的全部文件路径和注释文件的全部文件路径
data_file_path_list = []
annotations_file_path_list = []
for subject_dir in subject_dirs_list:
    #EEG信号文件列表
    data_file_path = [os.path.join(subject_dir, f) for f in os.listdir(subject_dir) if f.endswith('.edf')]
    data_file_path_list.extend(data_file_path)

    #注释文件列表
    annotations_file_path = [os.path.join(subject_dir, f) for f in os.listdir(subject_dir) if f.endswith('.event')]
    annotations_file_path_list.extend(annotations_file_path)

In [11]:

annotation = wfdb.rdann(annotations_file_path_list[2], "")
annotation_list = annotation.aux_note
annotation_list


['T2 duration: 4.1',
 'T0 duration: 4.2',
 'T1 duration: 4.1',
 'T0 duration: 4.2',
 'T1 duration: 4.1',
 'T0 duration: 4.2',
 'T2 duration: 4.1',
 'T0 duration: 4.2',
 'T2 duration: 4.1',
 'T0 duration: 4.2',
 'T1 duration: 4.1',
 'T0 duration: 4.2',
 'T1 duration: 4.1',
 'T0 duration: 4.2',
 'T2 duration: 4.1',
 'T0 duration: 4.2',
 'T1 duration: 4.1',
 'T0 duration: 4.2',
 'T2 duration: 4.1',
 'T0 duration: 4.2',
 'T2 duration: 4.1',
 'T0 duration: 4.2',
 'T1 duration: 4.1',
 'T0 duration: 4.2',
 'T1 duration: 4.1',
 'T0 duration: 4.2',
 'T2 duration: 4.1',
 'T0 duration: 4.2',
 'T1 duration: 4.1']

# 2.循环切片EEG信号并保存为json文件(使用时需要修改数据集根目录)

In [28]:
#根据文件名和Label获取文件夹名字
def get_dir_name(file_name, label):
    file_name = file_name.split("R")[-1] #Task number

    #Task 1
    if file_name == "03":
        if label == "T0":
            return "T0"
        if label == "T1":
            return "T1_Movement_Left_Fist"
        if label == "T2":
            return "T2_Movement_Right_Fist"
    #Task 2
    if file_name == "04":
        if label == "T0":
            return "T0"
        if label == "T1":
            return "T1_Imagery_Left_Fist"
        if label == "T2":
            return "T2_Imagery_Right_Fist"
    #Task 3
    if file_name == "05":
        if label == "T0":
            return "T0"
        if label == "T1":
            return "T1_Movement_Both_Fists"
        if label == "T2":
            return "T2_Movement_Both_Feet"
    #Task 4
    if file_name == "06":
        if label == "T0":
            return "T0"
        if label == "T1":
            return "T1_Imagery_Both_Fists"
        if label == "T2":
            return "T2_Imagery_Both_Feet"
    #Task 1        
    if file_name == "07":
        if label == "T0":
            return "T0"
        if label == "T1":
            return "T1_Movement_Left_Fist"
        if label == "T2":
            return "T2_Movement_Right_Fist"
    #Task 2
    if file_name == "08":
        if label == "T0":
            return "T0"
        if label == "T1":
            return "T1_Imagery_Left_Fist"
        if label == "T2":
            return "T2_Imagery_Right_Fist"
    #Task 3
    if file_name == "09":
        if label == "T0":
            return "T0"
        if label == "T1":
            return "T1_Movement_Both_Fists"
        if label == "T2":
            return "T2_Movement_Both_Feet"
    #Task 4
    if file_name == "10":
        if label == "T0":
            return "T0"
        if label == "T1":
            return "T1_Imagery_Both_Fists"
        if label == "T2":
            return "T2_Imagery_Both_Feet"
    #Task 1
    if file_name == "11":
        if label == "T0":
            return "T0"
        if label == "T1":
            return "T1_Movement_Left_Fist"
        if label == "T2":
            return "T2_Movement_Right_Fist"
    #Task 2
    if file_name == "12":
        if label == "T0":
            return "T0"
        if label == "T1":
            return "T1_Imagery_Left_Fist"
        if label == "T2":
            return "T2_Imagery_Right_Fist"
    #Task 3
    if file_name == "13":
        if label == "T0":
            return "T0"
        if label == "T1":
            return "T1_Movement_Both_Fists"
        if label == "T2":
            return "T2_Movement_Both_Feet"
    #Task 4
    if file_name == "14":
        if label == "T0":
            return "T0"
        if label == "T1":
            return "T1_Imagery_Both_Fists"
        if label == "T2":
            return "T2_Imagery_Both_Feet"

get_dir_name("S001R14", "T1")


'T1_Imagery_Both_Fists'

In [29]:
#保存数据集根目录
save_root_path = "D:\\BCI2100 Classed Dataset\\Original Data"

index = 1
for data_file_path in data_file_path_list:
    #读取EDF+文件信息
    f = pyedflib.EdfReader(data_file_path)
    channel_num = f.signals_in_file #通道数
    channel_list = f.getSignalLabels() #通道名
    signal_headers = f.getSignalHeaders() #通道注释

    #将信号转换为矩阵
    raw_eeg = mne.io.read_raw_edf(data_file_path, preload=True)
    raw_eeg = raw_eeg.to_data_frame()
    raw_eeg = raw_eeg.values
    raw_eeg = raw_eeg[:, 1:] #去除第一列时间点信息
    raw_eeg = np.transpose(raw_eeg) #转置之后每行为单通道EEG信号  

    #获取当前EEG信号对应的注释文件
    annotation = wfdb.rdann(data_file_path, "event")
    annotation_list = annotation.aux_note
    if annotation_list == []:
        continue #跳过每个受试者的前两组一分钟基线实验数据

    #当前文件名
    name = data_file_path.split("\\")[-1]
    name = name.split(".")[0]

    #对整个EEG信号进行切片并归类
    start_point = 0 #起始时间点
    for slice in annotation_list:
        #当前切片的信息
        label = slice.split(" ")[0] #类别标签（T0, T1, T2)
        sample_len = eval(slice.split(" ")[2]) * 160 #样本点个数 = 持续时间 * 采样率（采样率160Hz）

        #对EEG矩阵进行切片
        raw_eeg_patch = raw_eeg[:, start_point : int(start_point+sample_len)]
        raw_eeg_patch = raw_eeg_patch.tolist()

        #获取文件夹名字
        dir_name = get_dir_name(name ,label)

        #对文件进行分类存储
        file_name = f'{label}_{name}_{start_point}_{int(start_point+sample_len-1)}.json' #文件名信息：1.标签 2.受试者 3.Task 4.起始终止样本点
        file_path = os.path.join(save_root_path, dir_name) #文件夹名称
        file_path = os.path.join(file_path, file_name) #完整路径
        with open(file_path, 'w') as f:
            json.dump(raw_eeg_patch, f)

        #下一轮的起始点
        start_point = int(start_point+sample_len)

    #进度条
    print(index / len(data_file_path_list))
    index = index + 1





Extracting EDF parameters from D:\eeg-motor-movementimagery-dataset-1.0.0\files\S001\S001R01.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 9759  =      0.000 ...    60.994 secs...
Extracting EDF parameters from D:\eeg-motor-movementimagery-dataset-1.0.0\files\S001\S001R02.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 9759  =      0.000 ...    60.994 secs...
Extracting EDF parameters from D:\eeg-motor-movementimagery-dataset-1.0.0\files\S001\S001R03.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...
0.000655307994757536
Extracting EDF parameters from D:\eeg-motor-movementimagery-dataset-1.0.0\files\S001\S001R04.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...
0.001310615989515072
Extr

C:\Users\CYQ\AppData\Local\Temp\ipykernel_97008\821579309.py:13: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw_eeg = mne.io.read_raw_edf(data_file_path, preload=True)


0.7791612057667103
Extracting EDF parameters from D:\eeg-motor-movementimagery-dataset-1.0.0\files\S100\S100R04.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 15743  =      0.000 ...   122.992 secs...


C:\Users\CYQ\AppData\Local\Temp\ipykernel_97008\821579309.py:13: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw_eeg = mne.io.read_raw_edf(data_file_path, preload=True)


0.7798165137614679
Extracting EDF parameters from D:\eeg-motor-movementimagery-dataset-1.0.0\files\S100\S100R05.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 15743  =      0.000 ...   122.992 secs...


...

0.7850589777195282
Extracting EDF parameters from D:\eeg-motor-movementimagery-dataset-1.0.0\files\S100\S100R13.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 15743  =      0.000 ...   122.992 secs...


C:\Users\CYQ\AppData\Local\Temp\ipykernel_97008\821579309.py:13: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw_eeg = mne.io.read_raw_edf(data_file_path, preload=True)


0.7857142857142857
Extracting EDF parameters from D:\eeg-motor-movementimagery-dataset-1.0.0\files\S100\S100R14.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 15743  =      0.000 ...   122.992 secs...


C:\Users\CYQ\AppData\Local\Temp\ipykernel_97008\821579309.py:13: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw_eeg = mne.io.read_raw_edf(data_file_path, preload=True)


0.7863695937090432
Extracting EDF parameters from D:\eeg-motor-movementimagery-dataset-1.0.0\files\S101\S101R01.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 9759  =      0.000 ...    60.994 secs...
Extracting EDF parameters from D:\eeg-motor-movementimagery-dataset-1.0.0\files\S101\S101R02.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 9759  =      0.000 ...    60.994 secs...
Extracting EDF parameters from D:\eeg-motor-movementimagery-dataset-1.0.0\files\S101\S101R03.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...
0.7870249017038008
Extracting EDF parameters from D:\eeg-motor-movementimagery-dataset-1.0.0\files\S101\S101R04.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...
0.787680